In [2]:
#spark imports
import findspark
findspark.init()
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
from pyspark.ml.feature import IndexToString


In [3]:
spark = SparkSession.builder.appName("recommendation_system").config("spark.executor.memory", "20g").getOrCreate()

In [4]:
# products data up to 2014
product_data = spark.read.json('./metadata.json.gz')

# ratings of the products
rating_data = spark.read.csv('./amazon_product_rating.csv')

# only books ratings
books_rating_data = spark.read.csv('./ratings_Books.csv')

In [5]:
product_data.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- imUrl: string (nullable = true)
 |-- price: double (nullable = true)
 |-- related: struct (nullable = true)
 |    |-- also_bought: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- also_viewed: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- bought_together: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- buy_after_viewing: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- salesRank: struct (nullable = true)
 |    |-- Appliances: long (nullable = true)
 |    |-- Arts, Crafts & Sewing: long (nullable = true)
 |    |-- Automotive: long

In [6]:
rating_data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



In [7]:
books_rating_data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



In [8]:
product_data.show(10)

+---------------+----------+----------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|_corrupt_record|      asin|     brand|          categories|         description|               imUrl|price|             related|           salesRank|               title|
+---------------+----------+----------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|           null|0001048791|      null|           [[Books]]|                null|http://ecx.images...| null|                null|{null, null, null...|The Crucible: Per...|
|           null|0000143561|      null|[[Movies & TV, Mo...|3Pack DVD set - I...|http://g-ecx.imag...|12.99|{null, [B0036FO6S...|{null, null, null...|Everyday Italian ...|
|           null|0000037214|Big Dreams|[[Clothing, Shoes...|                null|http://ecx.images...| 6.99|{null, [B00JO8II7...|{null, null

In [9]:
books_rating_data.show(10)

+--------------+----------+---+----------+
|           _c0|       _c1|_c2|       _c3|
+--------------+----------+---+----------+
| AH2L9G3DQHHAJ|0000000116|4.0|1019865600|
|A2IIIDRK3PRRZY|0000000116|1.0|1395619200|
|A1TADCM7YWPQ8M|0000000868|4.0|1031702400|
| AWGH7V0BDOJKB|0000013714|4.0|1383177600|
|A3UTQPQPM4TQO0|0000013714|5.0|1374883200|
| A8ZS0I5L5V31B|0000013714|5.0|1393632000|
| ACNGUPJ3A3TM9|0000013714|4.0|1386028800|
|A3BED5QFJWK88M|0000013714|4.0|1350345600|
|A2SUAM1J3GNN3B|0000013714|5.0|1252800000|
| APOZ15IEYQRRR|0000013714|5.0|1362787200|
+--------------+----------+---+----------+
only showing top 10 rows



In [10]:
def getRatingData(df):
    data_schema = StructType([
    StructField("product_id", StringType(), True),
    StructField("client_id", StringType(), True),
    StructField("rating", FloatType(), True),
    StructField("timestamp", LongType(), True)])

    df = df.withColumnRenamed("_c0", "client_id")
    df = df.withColumnRenamed("_c1", "product_id")
    df = df.withColumnRenamed("_c2", "rating")
    df = df.withColumnRenamed("_c3", "timestamp")
    
    cols = [field.name for field in data_schema]
    spark_df = df.select(*cols)

    return spark_df

In [11]:
books_rating = getRatingData(books_rating_data)

In [12]:
books_rating.show(5)

+----------+--------------+------+----------+
|product_id|     client_id|rating| timestamp|
+----------+--------------+------+----------+
|0000000116| AH2L9G3DQHHAJ|   4.0|1019865600|
|0000000116|A2IIIDRK3PRRZY|   1.0|1395619200|
|0000000868|A1TADCM7YWPQ8M|   4.0|1031702400|
|0000013714| AWGH7V0BDOJKB|   4.0|1383177600|
|0000013714|A3UTQPQPM4TQO0|   5.0|1374883200|
+----------+--------------+------+----------+
only showing top 5 rows



In [13]:
books_rating.count()

22507155

In [14]:
def check_for_nulls(df):
    df = df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

    return df

check_for_nulls(books_rating)

+----------+---------+------+---------+
|product_id|client_id|rating|timestamp|
+----------+---------+------+---------+
|         0|        0|     0|        0|
+----------+---------+------+---------+



In [15]:
books_rating = books_rating.withColumn("rating", col("rating").cast("float"))

In [16]:
books_rating.groupBy('product_id').count().orderBy(desc('count')).show(20)

+----------+-----+
|product_id|count|
+----------+-----+
|0439023483|21398|
|030758836X|19867|
|0439023513|14114|
|0385537859|12973|
|0007444117|12629|
|0375831002|12571|
|038536315X|12564|
|0345803485|12290|
|0316055433|11746|
|0849922070|10424|
|0007442920|10172|
|0345803493| 9980|
|0399159347| 9906|
|0345803507| 9610|
|0007386648| 9062|
|1469984202| 8280|
|0606238409| 8106|
|0316044695| 7986|
|0141039280| 7905|
|0545265355| 7786|
+----------+-----+
only showing top 20 rows



In [17]:
top_clients = books_rating.groupBy("client_id").agg(count("rating").alias("total_ratings")) \
                .orderBy(desc("total_ratings")).limit(20)

top_clients.show()

+--------------+-------------+
|     client_id|total_ratings|
+--------------+-------------+
|A14OJS0VWMOSWO|        43201|
|   AFVQZQ8PW0L|        28816|
|A2F6N60Z96CAJI|         6121|
| A320TMDV6KCFU|         5955|
|A2OJW07GQRNJUT|         5443|
|A328S9RN3U5M68|         5031|
|A1S3C5OFU508P3|         4599|
|A13QTZ8CIMHHG4|         4564|
|A1X8VZWTOG8IS6|         4050|
| AHD101501WCN1|         4044|
|A2VKWLCNZF4ZVB|         4023|
|A1K1JW1C5CUSUZ|         3935|
|A2TX179XAT5GRP|         3879|
|A1M8PP7MLHNBQB|         3789|
|A1NATT3PN24QWY|         3716|
|A21NVBFIEQWDSG|         3554|
|A2EDZH51XHFA9B|         3390|
| AHUT55E980RDR|         3183|
|A281NPSIMI1C2R|         2740|
|A13G1TKIKHGV3F|         2683|
+--------------+-------------+



In [18]:
def get_distribution(df, column) -> DataFrame:
    rating_distribution = df.groupBy(column).agg(count(column).alias("count"), 
                                        round(count(column)/df.count()*100, 2).alias("percentage"))

    rating_distribution.show()

In [19]:
get_distribution(books_rating, "rating")

+------+--------+----------+
|rating|   count|percentage|
+------+--------+----------+
|   5.0|13886788|      61.7|
|   2.0|  978560|      4.35|
|   3.0| 1922390|      8.54|
|   1.0| 1116862|      4.96|
|   4.0| 4602555|     20.45|
+------+--------+----------+



In [20]:
sampled_data = books_rating.sample(fraction=0.01, seed=42)

In [21]:
get_distribution(sampled_data, "rating")

+------+------+----------+
|rating| count|percentage|
+------+------+----------+
|   5.0|139374|     61.62|
|   2.0|  9891|      4.37|
|   3.0| 19369|      8.56|
|   1.0| 10983|      4.86|
|   4.0| 46561|     20.59|
+------+------+----------+



In [22]:
# Create a list of column names to index
columns_to_index = list(set(sampled_data.columns) - set(['rating']))

# Create a list of StringIndexer objects
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in columns_to_index]

# Create a pipeline with all the indexers
pipeline = Pipeline(stages=indexers)

# Fit the pipeline to the data
model = pipeline.fit(sampled_data)

# Transform the data with the fitted model
transformed = model.transform(sampled_data)

In [23]:
# Show the first 10 rows of the transformed data
transformed.show(10)

+----------+--------------+------+----------+---------------+----------------+---------------+
|product_id|     client_id|rating| timestamp|timestamp_index|product_id_index|client_id_index|
+----------+--------------+------+----------+---------------+----------------+---------------+
|000100039X|A3UZYPFH8DC9XW|   3.0|1394236800|          281.0|          6753.0|       153682.0|
|000100039X|A2MPRPCAQLTR3L|   5.0|1310688000|         1049.0|          6753.0|         8873.0|
|000100039X| ANWMXAZZR2J0A|   5.0|1367625600|          176.0|          6753.0|       182105.0|
|0001473123|A2UF7RXEPL9ZJI|   5.0|1371859200|          567.0|         29856.0|       104280.0|
|0002007770|A2HQWMWL2WZWST|   5.0|1359590400|          402.0|            18.0|        87115.0|
|0002007770| AA9ATGCL96YL2|   2.0|1158105600|         3846.0|            18.0|       163806.0|
|0002007770| ASO6C0E3ELPSQ|   1.0|1304985600|         1454.0|            18.0|       188576.0|
|0002007770|A3E432LKN617C0|   4.0|1287705600|     

In [24]:
als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="client_id_index",itemCol="product_id_index",ratingCol="rating",coldStartStrategy="drop",nonnegative=True)
model=als.fit(transformed)

In [25]:
model.itemFactors.count()

151991

In [26]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
predictions=model.transform(transformed)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))
predictions.show()

RMSE=0.04042688162149142
+----------+--------------+------+----------+---------------+----------------+---------------+----------+
|product_id|     client_id|rating| timestamp|timestamp_index|product_id_index|client_id_index|prediction|
+----------+--------------+------+----------+---------------+----------------+---------------+----------+
|0805093079|A12TW0NZMHVIIE|   5.0|1360713600|          201.0|            26.0|        18392.0|   4.89166|
|0805093079|A19Q81JSBFTVSM|   5.0|1368230400|          487.0|            26.0|        27657.0|   4.89166|
|0805093079|A1NELE68WF76CW|   5.0|1360454400|          142.0|            26.0|        46378.0|   4.89166|
|0805093079|A2G2J7SOUUAX1I|   1.0|1321660800|         1142.0|            26.0|        84828.0|  0.978332|
|0805093079| A37V1RSR4Z6R2|   5.0|1321315200|         1141.0|            26.0|       122323.0|   4.89166|
|0805093079| A44JLRQJ4XY49|   5.0|1325721600|          746.0|            26.0|       155551.0|   4.89166|
|0805093079| AG34WV90

In [27]:
ver = model.uid
ts = unix_timestamp(current_timestamp())
product_vectors = model.itemFactors.select("id",\
                                        col("features").alias("model_factor"),\
                                        lit(ver).alias("model_version"),\
                                        ts.alias("model_timestamp"))
product_vectors.show(5)

+---+--------------------+----------------+---------------+
| id|        model_factor|   model_version|model_timestamp|
+---+--------------------+----------------+---------------+
|  0|[0.020638432, 0.0...|ALS_991d3a6867c9|     1677619897|
| 10|[0.01492299, 0.01...|ALS_991d3a6867c9|     1677619897|
| 20|[1.068503, 0.0077...|ALS_991d3a6867c9|     1677619897|
| 30|[0.017486217, 0.0...|ALS_991d3a6867c9|     1677619897|
| 40|[0.53731984, 1.12...|ALS_991d3a6867c9|     1677619897|
+---+--------------------+----------------+---------------+
only showing top 5 rows



In [28]:
product_index_meta = [
    f.metadata for f in transformed.schema.fields if f.name == "product_id_index"]
product_index_labels = product_index_meta[0]["ml_attr"]["vals"]

reviewerId_converter = IndexToString(inputCol="id", outputCol="product",   labels=product_index_labels)
PredictedLabels = reviewerId_converter.transform(product_vectors)
PredictedLabels = PredictedLabels.drop('id')
PredictedLabels.show(10)

+--------------------+----------------+---------------+----------+
|        model_factor|   model_version|model_timestamp|   product|
+--------------------+----------------+---------------+----------+
|[0.020638432, 0.0...|ALS_991d3a6867c9|     1677619919|0439023483|
|[0.01492299, 0.01...|ALS_991d3a6867c9|     1677619919|0007442920|
|[1.068503, 0.0077...|ALS_991d3a6867c9|     1677619919|0606238409|
|[0.017486217, 0.0...|ALS_991d3a6867c9|     1677619919|1442362383|
|[0.53731984, 1.12...|ALS_991d3a6867c9|     1677619919|043935806X|
|[0.9242187, 1.629...|ALS_991d3a6867c9|     1677619919|0425263916|
|[0.9700693, 1.187...|ALS_991d3a6867c9|     1677619919|0307749649|
|[0.75801194, 0.34...|ALS_991d3a6867c9|     1677619919|074356619X|
|[0.02503104, 0.14...|ALS_991d3a6867c9|     1677619919|0449014509|
|[0.3117248, 0.222...|ALS_991d3a6867c9|     1677619919|1469241943|
+--------------------+----------------+---------------+----------+
only showing top 10 rows



In [29]:
PredictedLabels.count()

151991

In [30]:
product_data_df = product_data.dropDuplicates(subset=["asin"])

In [31]:
def getMetaData(df):

    spark_df = df.select(
    "asin", 
    "categories",
    "brand", 
    "description", 
    "imUrl", 
    "price", 
    col("related.also_bought").alias("also_bought"),
    col("related.also_viewed").alias("also_viewed"),
    col("related.bought_together").alias("bought_together"),
    col("related.buy_after_viewing").alias("buy_after_viewing"),
    "title",
)

    return spark_df

In [32]:
product_data = getMetaData(product_data_df)

In [33]:
product_data.show(10)

+----------+----------+-----+--------------------+--------------------+-----+--------------------+--------------------+---------------+--------------------+--------------------+
|      asin|categories|brand|         description|               imUrl|price|         also_bought|         also_viewed|bought_together|   buy_after_viewing|               title|
+----------+----------+-----+--------------------+--------------------+-----+--------------------+--------------------+---------------+--------------------+--------------------+
|0001837192| [[Books]]| null|                null|http://ecx.images...| null|                null|                null|           null|                null|Pieter Brueghel's...|
|0001845357| [[Books]]| null|Great granddaught...|http://ecx.images...| 6.15|                null|                null|           null|                null|    Ballad of Favour|
|0002216973| [[Books]]| null|                null|http://ecx.images...| null|                null|[0812823354,

In [34]:
#product_data.groupBy("main_category").agg(count("*").alias("count")).orderBy(col("count").desc()).show(50)

In [35]:
book_data = product_data.filter(product_data.categories[0][0] == 'Books')

In [36]:
book_data.count()

2369958

In [37]:
check_for_nulls(book_data)

+----+----------+-------+-----------+-----+------+-----------+-----------+---------------+-----------------+------+
|asin|categories|  brand|description|imUrl| price|also_bought|also_viewed|bought_together|buy_after_viewing| title|
+----+----------+-------+-----------+-----+------+-----------+-----------+---------------+-----------------+------+
|   0|         0|2369847|    1248624|50867|691033|    1101405|    2066517|        1822528|          1124916|431194|
+----+----------+-------+-----------+-----+------+-----------+-----------+---------------+-----------------+------+



In [38]:
def drop_columns(df: DataFrame, columns_to_drop: list) -> DataFrame:
    for col in columns_to_drop:
        df = df.drop(col)
    return df

In [39]:
book_data = drop_columns(book_data, ['description', 'brand', 'price', 'also_bought', 'also_viewed', 'bought_together', 'buy_after_viewing'])

In [40]:
book_data = book_data.dropna(how='any')
book_data.count()

1890214

In [41]:
sampled_books = book_data.sample(fraction=0.1, seed=42)

In [42]:
sampled_books.count()

189635

In [43]:
sampled_books.show(5)

+----------+----------+--------------------+--------------------+
|      asin|categories|               imUrl|               title|
+----------+----------+--------------------+--------------------+
|0002000946| [[Books]]|http://ecx.images...|       Divine Hunger|
|0002251213| [[Books]]|http://ecx.images...|Ronald Reagan: Th...|
|0002252104| [[Books]]|http://ecx.images...|Grains (Gourmet P...|
|0006387403| [[Books]]|http://ecx.images...|The Pale Abyssini...|
|0006514685| [[Books]]|http://ecx.images...|The Essential Cli...|
+----------+----------+--------------------+--------------------+
only showing top 5 rows



In [44]:
sampled_books = sampled_books.withColumnRenamed("asin", "product_id")

@udf(returnType=StringType())
def extract_category(categories):
    return str(categories[0][0])

sampled_books = sampled_books.withColumn("category", extract_category(sampled_books.categories))
sampled_books = drop_columns(sampled_books, ["categories"])

In [119]:
book_data = book_data.withColumnRenamed("asin", "product_id")

@udf(returnType=StringType())
def extract_category(categories):
    return str(categories[0][0])

book_data = book_data.withColumn("category", extract_category(book_data.categories))
book_data = drop_columns(book_data, ["categories"])

In [45]:
sampled_books.show(5)

+----------+--------------------+--------------------+--------+
|product_id|               imUrl|               title|category|
+----------+--------------------+--------------------+--------+
|0002000946|http://ecx.images...|       Divine Hunger|   Books|
|0002251213|http://ecx.images...|Ronald Reagan: Th...|   Books|
|0002252104|http://ecx.images...|Grains (Gourmet P...|   Books|
|0006387403|http://ecx.images...|The Pale Abyssini...|   Books|
|0006514685|http://ecx.images...|The Essential Cli...|   Books|
+----------+--------------------+--------------------+--------+
only showing top 5 rows



In [46]:
# assuming both dataframes have a column named "product_id"
matching_id = None

for id in sampled_books.select("product_id").rdd.flatMap(lambda x: x).collect():
    if PredictedLabels.filter(PredictedLabels.product == id).count() > 0:
        matching_id = id
        break

if matching_id:
    print("The first matching product_id is:", matching_id)
else:
    print("No matching product_id found.")

The first matching product_id is: 0020449313


In [111]:
def get_similar(the_id, num, sampled_books_df, predicted_labels_df, vector_col):
    """
    Given a item id, execute the recommendation script score query to find similar items,
    ranked by cosine similarity. We return the `num` most similar, excluding the item itself.
    """
    # Get the query vector
    query_vec = predicted_labels_df \
        .where(predicted_labels_df.product == the_id) \
        .select(vector_col) \
        .first()[vector_col]

    # Compute the cosine similarity between the query vector and all the other vectors
    cosine_sim = (predicted_labels_df \
        .select('product', vector_col) \
        .rdd \
        .map(lambda row: (row[0], cosine_similarity([query_vec], [row[1]])[0][0])) \
        .filter(lambda row: row[0] != the_id) \
        .sortBy(lambda row: row[1], ascending=False) \
        .take(num))

    # Get the details of the query product and the similar products
    book_asin = sampled_books_df \
        .where(sampled_books_df.product_id == the_id) \
        .select('product_id', 'title', 'category') \
        .first().asDict()

    similar_books = []
    for sim in cosine_sim:
        book = sampled_books_df \
            .where(sampled_books_df.product_id == sim[0]) \
            .select('product_id', 'title', 'category') \
            .first().asDict()
        book['similarity'] = sim[1]
        similar_books.append(book)

    return book_asin, similar_books

In [131]:
def display_similar(sampled_books_df, predicted_labels_df, the_id, vector_col, num=10):
    """
    Display query product, together with similar product and similarity scores, in a table
    """
    product, similar_books = get_similar(the_id, num, sampled_books_df, predicted_labels_df, vector_col)
    display(HTML("<h2>Get similar products for:</h2>"))
    display(HTML("<h4>%s (product_id - %s)</h4>" % (product['title'], product['product_id'])))
    display(HTML("<br>"))
    display(HTML("<h2>People who liked this product also liked these:</h2>"))
    sim_html = "<table border=0>"
    pd_data = []
    for rec in similar_books:
        r_score = rec['similarity']
        r_title = rec['title']
        r = {}
        r['product_id'] = rec['product_id']
        r['title'] = r_title
        r['similarity'] = r_score
        pd_data.append(r)
        sim_html += "<tr><td><h5>%s</h5></td><td><h5>%2.3f</h5></td></tr>" % (r_title, r_score)
    sim_html += "</table>"
    pd.set_option('display.max_colwidth', -1) 
    pd_df = pd.DataFrame (pd_data)
    display(HTML(pd_df.to_html()))
    display(HTML(sim_html))

In [133]:
from IPython.display import Image, HTML, display
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
data = display_similar(book_data,PredictedLabels, '0020449313', num=10, vector_col='model_factor')

c:\Users\Dell\anaconda3\envs\sparkenv\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,product_id,title,similarity
0,0064404455,Tom's Midnight Garden,1.000000
1,1563637480,PDR: Physicians Desk Reference 2010 (Physicians' Desk Reference (Pdr)),0.903649
2,0765309386,Spin,0.897177
3,1465203923,Experiments in General Chemistry,0.891805
4,0231148518,In Their Siblings' Voices: White Non-Adopted Siblings Talk About Their Experiences Being Raised with Black and Biracial Brothers and Sisters,0.889620
5,1594733007,The Lectio Divina - The Sacred Art: Transforming Words &amp; Images into Heart-Centered Prayer (The Art of Spiritual Living),0.888772
6,0137016611,"Your Credit Score, Your Money &amp; What's at Stake (Updated Edition): How to Improve the 3-Digit Number that Shapes Your Financial Future",0.888498
7,1463566239,"TUTORING: Complete Home Business Guide: Tutor at home, Set your own Fees, Set your own schedule, Earn more tutoring online, tutor to international people",0.882772
8,0984013016,Haymarket Square,0.882402
9,0451211979,Eat Cake,0.872774


Tom's Midnight Garden,1.000
PDR: Physicians Desk Reference 2010 (Physicians' Desk Reference (Pdr)),0.904
Spin,0.897
Experiments in General Chemistry,0.892
In Their Siblings' Voices: White Non-Adopted Siblings Talk About Their Experiences Being Raised with Black and Biracial Brothers and Sisters,0.890
The Lectio Divina - The Sacred Art: Transforming Words & Images into Heart-Centered Prayer (The Art of Spiritual Living),0.889
"Your Credit Score, Your Money & What's at Stake (Updated Edition): How to Improve the 3-Digit Number that Shapes Your Financial Future",0.888
"TUTORING: Complete Home Business Guide: Tutor at home, Set your own Fees, Set your own schedule, Earn more tutoring online, tutor to international people",0.883
Haymarket Square,0.882
Eat Cake,0.873


In [114]:

query_vec = PredictedLabels \
        .where(PredictedLabels.product == '0020449313') \
        .select('model_factor') \
        .first()['model_factor']


cosine_sim = (PredictedLabels
        .select('product', 'model_factor')
        .rdd
        .map(lambda row: (row[0], cosine_similarity([query_vec], [row[1]])[0][0]))
        .filter(lambda row: row[0] != '0020449313')
        .sortBy(lambda row: row[1], ascending=False)
        .take(10))

In [115]:
book_asin = sampled_books \
        .where(sampled_books.product_id == '0020449313') \
        .select('product_id', 'title', 'category') \
        .first().asDict()

In [121]:
similar_books = []
for sim in cosine_sim:
        book = book_data \
        .where(book_data.product_id == sim[0]) \
        .select('product_id', 'title', 'category') \
        .first().asDict()
        book['similarity'] = sim[1]
        similar_books.append(book)

In [130]:
similar_books

[{'product_id': '0064404455',
  'title': "Tom's Midnight Garden",
  'category': 'Books',
  'similarity': 1.0000000000000002},
 {'product_id': '1563637480',
  'title': "PDR: Physicians Desk Reference 2010 (Physicians' Desk Reference (Pdr))",
  'category': 'Books',
  'similarity': 0.9036492562205621},
 {'product_id': '0765309386',
  'title': 'Spin',
  'category': 'Books',
  'similarity': 0.8971766225597146},
 {'product_id': '1465203923',
  'title': 'Experiments in General Chemistry',
  'category': 'Books',
  'similarity': 0.8918050897412294},
 {'product_id': '0231148518',
  'title': "In Their Siblings' Voices: White Non-Adopted Siblings Talk About Their Experiences Being Raised with Black and Biracial Brothers and Sisters",
  'category': 'Books',
  'similarity': 0.889620087237011},
 {'product_id': '1594733007',
  'title': 'The Lectio Divina - The Sacred Art: Transforming Words &amp; Images into Heart-Centered Prayer (The Art of Spiritual Living)',
  'category': 'Books',
  'similarity': 0